# Purpose of this notebook

See what kind of things we can find out from the [tweede kamer open data portal](https://opendata.tweedekamer.nl/).

There are two interfaces for this: an [Atom-style API called SyncFeed](https://opendata.tweedekamer.nl/documentatie/syncfeed-api), and an [OData API](https://opendata.tweedekamer.nl/documentatie/odata-api).

The Atom API is a little easier to speak yourself, OData can be a little more thorough but more work to use.

Either way, there is a [relational data model](https://opendata.tweedekamer.nl/documentatie/informatiemodel) that you should be thinking of.

Formats seem to be XML and JSON.

## Atom/SyncFeed API


In [3]:
#   Fetch all resources of the mentioned soort/category, save into a single .xml file of its name.
import pprint

import wetsuite.datacollect.tweedekamer_nl # contains some basic code dealing with this API
import wetsuite.helpers.etree

for soort in (#'Persoon', 
              'Fractie', 
              #'FractieZetel', 'FractieZetelPersoon',

              #'Kamerstukdossier',

              #Not used yet 
              #'Document',

              'Zaal', 
              #'Vergadering', 
              # 'Stemming', 
              #'Zaak',
              #'Verslag',
             ): # or wetsuite.datacollect.tweedekamer_nl.resource_types for all
    print( f' == {soort} ==' )

    # this function actually does little more stick that soort on a URL and fetch ...repeatedly if necessary
    etrees = wetsuite.datacollect.tweedekamer_nl.fetch_all( soort )     
    
    # it also returned a list of etree objects,  and because our plan is currently to write that into a single XML file, we want to merge that
    single_tree = wetsuite.datacollect.tweedekamer_nl.merge_etrees( etrees )  

    print( wetsuite.helpers.etree.debug_pretty( single_tree ) )

    #xmlstring = wetsuite.helpers.etree.tostring( single_tree )                # then save that 
    #with open('%s.xml'%soort, 'wb') as xf:   # technically injection-sensitive, except it's fine as long as we control the values
    #    xf.write( xmlstring )       


 == Fractie ==
<feed>
  <entry>
    <title>ea081a13-e90e-4b8c-a917-95b7026bb635</title>
    <id>https://gegevensmagazijn.tweedekamer.nl/SyncFeed/2.0/Entiteiten/ea081a13-e90e-4b8c-a917-95b7026bb635</id>
    <author>
      <name>Tweede Kamer der Staten-Generaal</name>
    </author>
    <updated>2023-08-29T11:23:19Z</updated>
    <category term="fractie"/>
    <link rel="next" href="https://gegevensmagazijn.tweedekamer.nl/SyncFeed/2.0/Feed?category=Fractie&amp;skiptoken=16662055"/>
    <content type="application/xml">
      <fractie id="ea081a13-e90e-4b8c-a917-95b7026bb635" bijgewerkt="2023-08-29T11:09:50Z" verwijderd="false">
        <nummer>2761</nummer>
        <afkorting>LPF</afkorting>
        <naamNl>Lijst Pim Fortuyn</naamNl>
        <naamEn>List Pim Fortuyn</naamEn>
        <aantalZetels nil="true"/>
        <aantalStemmen>541233</aantalStemmen>
        <datumActief>2002-05-23</datumActief>
        <datumInactief>2006-11-29</datumInactief>
      </fractie>
    </content>
  </entry

## OData interface

The SyncFeed API is perfectly functional, though it leaves you to do interpretation of the relations yourself, 
so let's see if the OData API is any more help.

There is a helpful library out there, [tkapi](https://github.com/openkamer/tkapi) (MIT license), that means we don't have to implement it ourselves.

Note that neither API is is a very efficient interface
when you want to sift through the entire collection of data in ways it was not initially designed for,
aldo why it might be valuable to make some datasets from this later.

...but this one _is_ a little more help along the way.

In [ ]:
!pip3 install tkapi

In [4]:
import tkapi, tkapi.document   
from tkapi.document import DocumentSoort
api = tkapi.TKApi()

In [10]:
# The document types are function calls, to wit:
list(name   for name in dir(api)   if name.startswith('get_'))

['get_activiteiten',
 'get_agendapunten',
 'get_all_items',
 'get_antwoorden',
 'get_besluiten',
 'get_commissies',
 'get_documenten',
 'get_dossiers',
 'get_fractie_zetels',
 'get_fracties',
 'get_geschenken',
 'get_item',
 'get_items',
 'get_kamervragen',
 'get_personen',
 'get_reizen',
 'get_related',
 'get_stemmingen',
 'get_vergaderingen',
 'get_verslagen',
 'get_verslagen_van_algemeen_overleg',
 'get_zaken']

In [11]:
dossiers = api.get_dossiers( )
len(dossiers)  # a few thousand, which is why this takes a few secdonds to fetch

6851

In [12]:
# Let's get a basic summary of dossiers. 
#    we find out apparently  dossier nummers  are not unique without the  toevoeging
for dossier in sorted(dossiers, key=lambda dossier:str(dossier.nummer)+(dossier.toevoeging or '')):   #[:200]: # show a few hundred, not all
    nummer_toevoeging = ('%s-%s'%(dossier.nummer, dossier.toevoeging or '')).rstrip('-')
    print( "== Dossier %s == %s =="%( nummer_toevoeging, dossier.titel) )
    print( '  ',dossier.url.replace(')','%29') ) # the replace is to make the notebook's url include the final bracket
    #for doci, document in enumerate(dossier.documenten):
    #    print( '   ',doci, 'bestand_url', document.bestand_url )

== Dossier 17050 == Misbruik en oneigenlijk gebruik op het gebied van belastingen, sociale zekerheid en subsidies ==
   https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Kamerstukdossier(58cf1611-70a0-436a-ba73-f5e0819fd2a9%29
== Dossier 17257 == Assemblee van de West-Europese Unie ==
   https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Kamerstukdossier(2af0d9a4-cccc-416b-870d-393e1299d540%29
== Dossier 18031 == Onderzoek militaire missie Suriname ==
   https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Kamerstukdossier(a77d0458-31d5-43ca-8b6d-3061ec8f8efe%29
== Dossier 18106 == Voortgang rivierdijkversterkingen ==
   https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Kamerstukdossier(3698cafd-4ffd-428c-9459-4aa205d98ef0%29
== Dossier 19291 == Parlementaire Assemblée van de NAVO ==
   https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Kamerstukdossier(b85f8944-7c73-47dc-8055-ed28bc901fc0%29
== Dossier 19637 == Vreemdelingenbeleid ==
   https://gegevensmagazijn.tweedekamer.n

In [14]:
# Let's say that our interest is finding what Raad van State has to say about wetsvoorstellen.
#   and also to find out what kind of documents there are in each dossier.
#
# We start by selecting dossiers where there already _is_ RvS advice.
#  - this is a decent filter for wetsvoorstellen
#  - and filters out wetsvoorstellen that don't need this advice (e.g. begroting)
# ...but we are about to find out    
# - there are other things that RVS advises on, like finances (see e.g. 36200) 
# - there are law changes that RVS does not advise on (e.g. TODO)


sorted_dossiers = sorted(dossiers, key=lambda d:d.nummer, reverse=True )

for dossier in sorted_dossiers:
    nummer_toevoeging = ('%s-%s'%(dossier.nummer, dossier.toevoeging or '')).rstrip('-')

    if (dossier.nummer%100) == 0: # ignore some specific special cases for now, because they're large
        continue

    sorted_docs = sorted(dossier.documenten, key=lambda d:d.volgnummer )
    has_raadvanstate = False
    for document in sorted_docs:
        try:
            # these come from an enum, try  list( tkapi.document.DocumentSoort )  to see a list
            if document.soort in (DocumentSoort.ADVIES_AFDELING_ADVISERING_RAAD_VAN_STATE, 
                                  #DocumentSoort.ADVIES_AFDELING_ADVISERING_RAAD_VAN_STATE_EN_NADER_RAPPORT, # seems to be begrotingstuff?  (TODO: check)
                                  DocumentSoort.ADVIES_AFDELING_ADVISERING_RAAD_VAN_STATE_EN_REACTIE_VAN_DE_INITIATIEFNEMERS,
                                ):
                has_raadvanstate = True
        except ValueError: # there's some invalid / non-covered soort values in the data
            pass # ignore
        # we can filter on more, but we may not need to?
    if not has_raadvanstate:
        continue
    
    # if it passed that, it's probably interesting to us.
    #if len(sorted_docs)>500:
    #    print( "\n\n== Dossier %s == %s =="%( dossier.nummer, dossier.titel) )
    #    print(' LARGE: %d documents'%len(sorted_docs))
    #    print(' %s ({{kamerdossier|%d}}'%(dossier.titel, dossier.nummer))
    #    continue

    print( "\n== %r == Dossier %s == %d docs == %s =="%( dossier.id, nummer_toevoeging, len(dossier.documenten), dossier.titel) )
    for document in sorted_docs:
        try:
            if 0: # just to make the summaries a little easier to read
                if document.soort in (DocumentSoort.MOTIE, DocumentSoort.AMENDEMENT, DocumentSoort.BRIEF_REGERING, DocumentSoort.VERSLAG_VAN_EEN_ALGEMEEN_OVERLEG,
                                    DocumentSoort.MEMORIE_VAN_TOELICHTING_INITIATIEFVOORSTEL,
                                    ):
                    continue
        except ValueError:
            print( "soort not known by tkapi")
            continue

        try:
            docsoort = document.soort
        except ValueError: # this seems to be internal inconsistency
            continue

        if True or docsoort in (DocumentSoort.ADVIES_AFDELING_ADVISERING_RAAD_VAN_STATE, 
                              DocumentSoort.ADVIES_AFDELING_ADVISERING_RAAD_VAN_STATE_EN_REACTIE_VAN_DE_INITIATIEFNEMERS):

            print( '#%s'%(document.volgnummer, ), document.soort.name)
            #print( 'soort', document.soort.name, '(%s)'%document.soort.value )
            print( '  onderwerp', document.onderwerp )     # for wetsvoorstel-dossiers, seems to often be the same as soort plus some detail (who a letter is from, who )
            print( '  citeertitel',document.titel_citeer ) # for wetsvoorstel-dossiers, this often seems to name the law. Or a related one, see e.g. 36195
            print( '  titel',document.titel )              # for wetsvoorstel-dossiers, this seems to often name the law, plus some reason
            #print( 'versies', document.versies )
            print( '  url', document.bestand_url )
            if 0:        # It may be interesting to know the document is part of multiple dossiers and/or multiple zaken
                print( '  zaken', document.zaken )
                #nums = document.dossier_nummers
                #nums.pop(dossier.nummer)
                #if len(nums)>0:
                #  print( "  also in dossiers: %s"%nums )

            print()


== '87a7ff74-bf81-4e71-a69e-2439ce536c2c' == Dossier 36346 == 5 docs == Voorstel van wet van het lid Van Houwelingen betreffende het houden van een raadplegend referendum over het Nederlandse lidmaatschap van de Europese Unie (Wet raadplegend referendum Nederlands EU-lidmaatschap) ==
#1 GELEIDENDE_BRIEF
  onderwerp Geleidende brief
  citeertitel Wet raadplegend referendum Nederlands EU-lidmaatschap
  titel Voorstel van wet van het lid Van Houwelingen betreffende het houden van een raadplegend referendum over het Nederlandse lidmaatschap van de Europese Unie (Wet raadplegend referendum Nederlands EU-lidmaatschap)
  url https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Document(2416f81d-4ee0-4111-a00b-c412992ed7ac)/TK.DA.GGM.OData.Resource()

#2 VOORSTEL_VAN_WET_INITIATIEFVOORSTEL
  onderwerp Voorstel van wet
  citeertitel Wet raadplegend referendum Nederlands EU-lidmaatschap
  titel Voorstel van wet van het lid Van Houwelingen betreffende het houden van een raadplegend referendum ov

KeyboardInterrupt: 